In [5]:
import sys
sys.path.append("../../..")

import pandas as pd
import random
import numpy as np

train_data = pd.read_csv("../../../data/ifly/train.csv")
valid_data = pd.read_csv("../../../data/ifly/valid.csv")
test_data = pd.read_csv("../../../data/ifly/test.csv")

train_data = train_data.rename(columns={'user_id': 'user_id'})
train_data = train_data.rename(columns={'question_id': 'item_id'})
train_data = train_data.rename(columns={'correct': 'score'})

valid_data = valid_data.rename(columns={'user_id': 'user_id'})
valid_data = valid_data.rename(columns={'question_id': 'item_id'})
valid_data = valid_data.rename(columns={'correct': 'score'})

test_data = test_data.rename(columns={'user_id': 'user_id'})
test_data = test_data.rename(columns={'question_id': 'item_id'})
test_data = test_data.rename(columns={'correct': 'score'})

sample_size = 6

def fake_train_data(train_data, valid_data, test_data, s):
    all_data = pd.concat([train_data, valid_data, test_data], axis=0)
    existing_all_users = all_data['user_id'].unique()
    existing_train_users = train_data['user_id'].unique()
    new_train_data = []
    for train_user in existing_train_users:
        user_data = train_data[train_data['user_id'] == train_user]

        num_rows = len(user_data)
        if num_rows < s:
            random_rows = list(range(num_rows))
            s = num_rows
        else:
            random_rows = random.sample(range(num_rows), s)

        fake_user_ids = []
        for _ in range(s):
            fake_user_id = max(existing_all_users) + 1
            while fake_user_id in existing_all_users:
                fake_user_id += 1
            existing_all_users = np.append(existing_all_users, fake_user_id)
            fake_user_ids.append(fake_user_id)

        pos = [(user_data.iloc[i]['score'] - 0.5) * 2 for i in random_rows]
        row_count = 0
        for index, row in user_data.iterrows():
            user_id = row['user_id']
            item_id = row['item_id']
            score = row['score']

            data_point = {
                'user_id': user_id,
                'item_id': item_id,
                'score': score,
                'user_id_pair': fake_user_ids,
                'pos': pos,
                'fake': 0
            }
            new_train_data.append(data_point)

            for i in range(s):   
                if row_count == random_rows[i]:
                    fake_score = 1 - score
                    fake_data_point = {
                        'user_id': fake_user_ids[i],
                        'item_id': item_id,
                        'score': fake_score,
                        'user_id_pair': [user_id] * s,
                        'pos': [- pos[i]] * s,
                        'fake': 1
                    }
                    new_train_data.append(fake_data_point)
                else:
                    fake_data_point = {
                        'user_id': fake_user_ids[i],
                        'item_id': item_id,
                        'score': score,
                        'user_id_pair': [user_id] * s,
                        'pos': [- pos[i]] * s,
                        'fake': 1
                    }
                    new_train_data.append(fake_data_point)
            row_count += 1

    new_train_data_df = pd.DataFrame(new_train_data)
    shuffled_data = new_train_data_df.sample(frac=1).reset_index(drop=True)
    return shuffled_data, max(existing_all_users)

train_data, user_num = fake_train_data(train_data, valid_data, test_data, sample_size)
train_data.to_csv("../../../data/ifly/" + str(sample_size) + "_fake_train.csv", index=False)

In [2]:
# coding: utf-8
# 2021/3/23 @ tongshiwei
import sys
sys.path.append("../../..")

import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

batch_size = 256
sample_size = 2

train_data = pd.read_csv("../../../data/ifly/" + str(sample_size) + "_fake_train.csv")
valid_data = pd.read_csv("../../../data/ifly/valid.csv")
test_data = pd.read_csv("../../../data/ifly/test.csv")

valid_data = valid_data.rename(columns={'user_id': 'user_id'})
valid_data = valid_data.rename(columns={'question_id': 'item_id'})
valid_data = valid_data.rename(columns={'correct': 'score'})

test_data = test_data.rename(columns={'user_id': 'user_id'})
test_data = test_data.rename(columns={'question_id': 'item_id'})
test_data = test_data.rename(columns={'correct': 'score'})



def transform(x, y, z, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

def transform_train_data(x, y, z, f, p, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32),
        torch.tensor(f, dtype=torch.int64),
        torch.tensor(p, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)



valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [valid_data, test_data]
]


user_id_pair = []
pos = []
for i, s in train_data.iterrows():
    codes1 = eval(s['user_id_pair'])
    codes2 = eval(s['pos'])
    user_id_pair.append(codes1)
    pos.append(codes2)
    
train = transform_train_data(train_data["user_id"], train_data["item_id"], train_data["score"], user_id_pair, pos, batch_size)
# print(train_data["pos"])
# print(max(train_data['user_id'].unique()))


/data/zhengz/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
sys.path.append("../../..")

import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

logging.getLogger().setLevel(logging.INFO)

user_num = max(train_data['user_id'].unique())
# print(user_num)
cdm = CIRT(user_num + 1, 12130, a_range=1, zeta=0.3)

cdm.train(train, valid, epoch=20, device="cuda")
cdm.save("cirt_ifly.params")

cdm.load("cirt_ifly.params")
rmse, mae, auc, accuracy = cdm.eval(test)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

Epoch 0: 100%|██████████| 3153/3153 [00:37<00:00, 84.60it/s] 


[Epoch 0] LogisticLoss: 0.811351


evaluating: 100%|██████████| 129/129 [00:00<00:00, 262.59it/s]


[Epoch 0] rmse: 0.497489, mae: 0.419493, auc: 0.624473, accuracy: 0.624302


Epoch 1: 100%|██████████| 3153/3153 [00:37<00:00, 85.20it/s] 


...

[Epoch 19] LogisticLoss: 0.434121


evaluating: 100%|██████████| 129/129 [00:00<00:00, 183.70it/s]
INFO:root:save parameters to cirt_ifly.params
INFO:root:load parameters from cirt_ifly.params


[Epoch 19] rmse: 0.396348, mae: 0.302146, auc: 0.839679, accuracy: 0.770459


evaluating: 100%|██████████| 322/322 [00:01<00:00, 194.75it/s]


rmse: 0.394825, mae: 0.300433, auc: 0.842306, accuracy: 0.772410


In [4]:
import logging
from EduCDM import CIRT
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import random
import numpy as np

batch_size = 256
def transform(x, y, z, batch_size, **params):
    x = x.to_numpy()
    y = y.to_numpy()
    z = z.to_numpy()
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)


test_data = pd.read_csv('../../../data/ifly/test.csv')
user_id_mapping = pd.read_csv('../../../data/ifly/user_id_mapping.csv')
test_data = test_data.merge(user_id_mapping, on='user_id', how='inner')

sorted_select_users = pd.read_csv('../../../data/ifly/sorted_select_users.csv')
sorted_select_users = sorted_select_users.drop("user_id_mapping", axis=1)
sorted_select_users = sorted_select_users.rename(columns={'user_id': 'user_id_mapping'})
test_data = test_data.merge(sorted_select_users, on='user_id_mapping', how='inner')

group_information = pd.read_csv('../../../data/ifly/group_information.csv')
group_information = group_information.drop("user_id", axis=1)
group_information = group_information.drop("count", axis=1)
test_data = test_data.merge(group_information, on='province_id', how='inner')


test1_data = test_data[test_data['avg'] < 80]
test2_data = test_data[test_data['avg'] >= 80]

test1, test2 = [
    transform(data["user_id"], data["question_id"], data["correct"], batch_size)
    for data in [test1_data, test2_data]
]
test1, test2

(<torch.utils.data.dataloader.DataLoader at 0x7fa0001a5750>,
 <torch.utils.data.dataloader.DataLoader at 0x7fa0001a7850>)

In [5]:
cdm.load("cirt_ifly.params")
rmse, mae, auc, accuracy = cdm.eval(test1)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

cdm.load("cirt_ifly.params")
rmse, mae, auc, accuracy = cdm.eval(test2)
print("rmse: %.6f, mae: %.6f, auc: %.6f, accuracy: %.6f" % (rmse, mae, auc, accuracy))

INFO:root:load parameters from cirt_ifly.params
evaluating: 100%|██████████| 143/143 [00:01<00:00, 99.72it/s] 
INFO:root:load parameters from cirt_ifly.params


rmse: 0.408366, mae: 0.316202, auc: 0.833092, accuracy: 0.753694


evaluating: 100%|██████████| 173/173 [00:01<00:00, 122.33it/s]

rmse: 0.382831, mae: 0.287054, auc: 0.842472, accuracy: 0.788208
